<a href="https://colab.research.google.com/github/SoudruhRomanCZ/KRIVANEK-240642-MLF/blob/main/PROJECT/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Import needed libraries

In [2]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

#1. data preprocesing
## 1.1 Importing data

In [7]:
train_data = pd.read_csv('label_train.csv')
test_data = pd.read_csv('test_format.csv')

In [ ]:
# Set the path to your data
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/MPA-MLF-Project_data'

# Load the CSV files
label_train = pd.read_csv(os.path.join(data_path, 'label_train.csv'))
test_format = pd.read_csv(os.path.join(data_path, 'test_format.csv'))

# Load all NPY files from the Train directory
train_data = []
for i in range(1370):  # Assuming files are named from 0.npy to 1369.npy
    file_path = os.path.join(data_path, 'Train', f'{i}.npy')
    if os.path.exists(file_path):
        data = np.load(file_path)
        train_data.append(data)

# Convert list to a NumPy array
train_data = np.array(train_data)

# Load all NPY files from the Test directory
test_data = []
for i in range(60):  # Assuming files are named from 0.npy to 59.npy
    file_path = os.path.join(data_path, 'Test', f'{i}.npy')
    if os.path.exists(file_path):
        data = np.load(file_path)
        test_data.append(data)

# Convert list to a NumPy array
test_data = np.array(test_data)

# Display the shapes of the loaded data
print(f'Train data shape: {train_data.shape}')
print(f'Test data shape: {test_data.shape}')


##1.2 Preprocesing data

In [9]:
# Normalize the data
train_data = train_data.astype('float32') / 255.0  # Assuming pixel values
test_data = test_data.astype('float32') / 255.0

# Preprocess the data
X = train_data.reshape(train_data.shape[0], -1)  # Flattening to 1D if needed
y = labels['label'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

AttributeError: 'DataFrame' object has no attribute 'reshape'

#2. Model Selection and Implementation

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes: 0, 1, 2

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#3. Model Training

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

#4. Model Evaluation

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

#5. Hyperparameter Tuning

In [ ]:
# Define a function to create the model for GridSearchCV
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Use KerasClassifier for GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'batch_size': [16, 32],
              'epochs': [50, 100],
              'optimizer': ['adam', 'rmsprop']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')

#6. Conclusion

